# Bus241a: Spring 2022
# Problem set 9 starter


This dataset is downloaded from Kaggle(https://www.kaggle.com/mirichoi0218/insurance)
Predict insurance costs based on the following variables:

* **age:** age of primary beneficiary
* **sex:** insurance contractor gender, female, male
* **bmi:** Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
* **children:** Number of children covered by health insurance / Number of dependents
* **smoker:** Smoking
* **region:** the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
* **charges:** Individual medical costs billed by health insurance

In [1]:
# Load tools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib
# Make plots appear in notebook
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import matplotlib as mpl

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

np.random.seed(42)

#### Read CSV file

In [7]:
import pandas as pd
import os
cwd = os.getcwd()
path='C:\\Users\\Li Lin\\Desktop'
os.chdir(path)

In [8]:
insurance_df = pd.read_csv('insurance.csv')

#### Check for missing values; Describe data

In [9]:
insurance_df.info()
print(insurance_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
(1338, 7)


In [10]:
insurance_df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [11]:
insurance_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [12]:
insurance_df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

## Create dummies

In [15]:
insuranceDummies_df = pd.get_dummies(insurance_df,drop_first=True)
insuranceDummies_df.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0


## Ordinal coding

In [13]:
insuranceOrd_df = insurance_df.copy()
replaceSex = {'male':1,'female':0}
replaceSmoke = {'no':0,'yes':1}
replaceRegion = {'southwest':0, 'southeast':1, 'northeast':2, 'northwest':3}
insuranceOrd_df['sex']= insuranceOrd_df['sex'].replace(replaceSex)
insuranceOrd_df['smoker']= insuranceOrd_df['smoker'].replace(replaceSmoke)
insuranceOrd_df['region']= insuranceOrd_df['region'].replace(replaceRegion)
insuranceOrd_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,0,16884.92400
1,18,1,33.770,1,0,1,1725.55230
2,28,1,33.000,3,0,1,4449.46200
3,33,1,22.705,0,0,3,21984.47061
4,32,1,28.880,0,0,3,3866.85520


#### Preparation

Generate X, y

In [20]:
X = insuranceDummies_df.drop('charges',axis=1)
y = insuranceDummies_df['charges']
XOrd = insuranceOrd_df.drop('charges',axis=1)

## Add yExtra as a new target

This is designed to add some more complicated interactions in the data to make things more interesting.  Some of these boost the impact of regional data, and one is a "dual interaction" between sex and region.  The regional data does not impact the mean in a linear fashion.  Region 2 is less than region 0.  No reason why this might not be the case in real data.  This modification is just trying to crank up the impact of region as a predictor.


In [21]:
yExtra = y + (XOrd['region']==0) * 1.*y.std() + ((XOrd['region']==2)) * -0.1*y.std() + (XOrd['sex']==1)*(XOrd['region']==3) * 3.*y.std()

## Part 1: Run linear regression on original data

Compare codings

Run a stadard monte-carlo cross validation using a linear regression for (X,y) and (Xord,y).  Do this with 100 monte-carlo runs, and test size = 0.25.  Report the mean train_score and test_score in each case.  Do your results differ much?


In [18]:
nmc = 100
lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=nmc, test_size=.25)
CVInfo = cross_validate(lr, X, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.7502133927697667
0.7478013913048929


In [23]:
nmc = 100
lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=nmc, test_size=.25)
CVInfo = cross_validate(lr, XOrd, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.7514628360603994
0.7427005422826383


## Part 2: Run linear regression on new data

Repeat the previous monte-carlo cross validation with a linear regression, but use the yExtra target.  In other words, run on (X,yExtra) and (Xord, yExtra).  

In [24]:
nmc = 100
lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=nmc, test_size=.25)
CVInfo = cross_validate(lr, X, yExtra, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.6500155815911798
0.6412143330955


In [25]:
nmc = 100
lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=nmc, test_size=.25)
CVInfo = cross_validate(lr, X, yExtra, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.649836154459436
0.6418158644681177


## Part 3: Ridge

Now explore cross validation with a Ridge regression on the (XOrd,yExta) pair. 

Set up a pipeline with the MinMaxScaler() and Ridge.  Run a GridSearchCV() using the following range for alpha.

[0.0001,0.001,0.01,0.05,0.1,0.25,0.5,1, 2]

Monte-carlo number is still 100, and testSize should be 0.25 as before.

Report our usual nice table across the values of alpha?  Any hope for Ridge fixing some of the problems of the ordinal coding?


In [47]:
fullModel = Pipeline([
    ("scaler",  MinMaxScaler()),
    ("Ridge", Ridge())
])
param_grid={'Ridge__alpha':[0.0001,0.001,0.01,0.05,0.1,0.25,0.5,1, 2]}
shuffle_split = ShuffleSplit(test_size=0.25, n_splits=100)

# set up search
grid_search=GridSearchCV(fullModel,param_grid,cv=shuffle_split, 
                              return_train_score=True)
# implement search
grid_search.fit(XOrd,yExtra)
# move results into DataFrame
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_Ridge__alpha']])

   rank_test_score  mean_test_score param_Ridge__alpha
0                9         0.427705             0.0001
1                8         0.427705              0.001
2                7         0.427706               0.01
3                6         0.427710               0.05
4                5         0.427715                0.1
5                4         0.427730               0.25
6                3         0.427753                0.5
7                2         0.427789                  1
8                1         0.427825                  2


In [48]:
# Print best params and model
print("best param:",grid_search.best_params_)
print("best model:",grid_search.best_estimator_)
print("best test score:",grid_search.best_score_)

best param: {'Ridge__alpha': 2}
best model: Pipeline(steps=[('scaler', MinMaxScaler()), ('Ridge', Ridge(alpha=2))])
best test score: 0.4278254589204915


The best test score for Ridge is almost the same as that of Linear Regression. I do not think there is hope for Ridge fixing some of the problems of the ordinal coding.

## Part 4: Kernel Ridge

Repeat the previous problem, but now use KernelRidge() again with (XOrd,yExtra)

Perform a GridSearchCV() using the following ranges,

alpha:
[0.0001,0.001,0.01,0.05,0.1,0.25,0.5]

gamma:
[0.01,0.05,0.25,0.5,1.]

Set the kernel to "rbf".

Again, number of monte-carlo's should be 100, and testSize as 0.25.  Again report a nice table across all the points in the grid.  Any improvements in your results?

In [50]:
from sklearn.kernel_ridge import KernelRidge
nlreg = KernelRidge(kernel='rbf')
fullModel = Pipeline([
    ("scaler", MinMaxScaler()),
    ("kridge", nlreg)
])

# set up dictionary for grid search
param_grid={'kridge__alpha':[0.0001,0.001,0.01,0.05,0.1,0.25,0.5],'kridge__gamma':[0.01,0.05,0.25,0.5,1.],'kridge__kernel':['rbf']}
# set up cross-validation shuffles
shuffle_split = ShuffleSplit(test_size=0.25, n_splits=100)
# set up search
grid_search=GridSearchCV(fullModel,param_grid,cv=shuffle_split, 
                              return_train_score=True,n_jobs=1)
# implement search
grid_search.fit(XOrd,yExtra)
# move results into DataFrame
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_kridge__alpha','param_kridge__gamma']])

    rank_test_score  mean_test_score param_kridge__alpha param_kridge__gamma
0                23         0.838257              0.0001                0.01
1                10         0.899150              0.0001                0.05
2                 4         0.905969              0.0001                0.25
3                15         0.888367              0.0001                 0.5
4                22         0.848630              0.0001                 1.0
5                26         0.801492               0.001                0.01
6                18         0.885575               0.001                0.05
7                 2         0.907773               0.001                0.25
8                 6         0.905202               0.001                 0.5
9                14         0.888780               0.001                 1.0
10               29         0.626587                0.01                0.01
11               24         0.833923                0.01                0.05

In [51]:
# Print best params and model
print("best param:",grid_search.best_params_)
print("best model:",grid_search.best_estimator_)
print("best test score:",grid_search.best_score_)

best param: {'kridge__alpha': 0.01, 'kridge__gamma': 0.5, 'kridge__kernel': 'rbf'}
best model: Pipeline(steps=[('scaler', MinMaxScaler()),
                ('kridge', KernelRidge(alpha=0.01, gamma=0.5, kernel='rbf'))])
best test score: 0.9078498137028015


Using KernelRidge() with XOrd,yExtra, there is a huge improvement from Linear and Ridge Regression.

## Part 5: Regression Tree

Now repeat your GridSearchCV() for a regression tree model.  Sweep across maximimum tree depths of 1-10.

Data for this part should be (X,yExtra).

Same parameters for the monte-carlo cross validation part of the grid search.

Print out the usual nice table across the tree depths.  Add mean_train_score to the table this time.


In [52]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
param_grid={'max_depth':range(1,11)}
# set up cross-validation shuffles
shuffle_split = ShuffleSplit(test_size=0.25, n_splits=100)
# set up search
grid_search=GridSearchCV(dtr,param_grid,cv=shuffle_split, 
                              return_train_score=True)
# implement search
grid_search.fit(X,yExtra)
# move results into DataFrame
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','mean_train_score','param_max_depth']])

# Print best params and model
print("best param:",grid_search.best_params_)
print("best model:",grid_search.best_estimator_)
print("best test score:",grid_search.best_score_)


   rank_test_score  mean_test_score  mean_train_score param_max_depth
0               10         0.290726          0.295163               1
1                9         0.467614          0.476967               2
2                8         0.802881          0.810755               3
3                3         0.903047          0.910493               4
4                1         0.913380          0.932337               5
5                2         0.909599          0.946315               6
6                4         0.895122          0.958352               7
7                5         0.880313          0.970658               8
8                6         0.870558          0.980319               9
9                7         0.861790          0.987429              10
best param: {'max_depth': 5}
best model: DecisionTreeRegressor(max_depth=5)
best test score: 0.9133804536435827


## Part 6: Now rerun tree with ordinal codes

Rerun exactly part 5, but use (XOrd,yExtra).  Is there much different from part 5?


In [53]:
dtr = DecisionTreeRegressor()
param_grid={'max_depth':range(1,11)}
# set up cross-validation shuffles
shuffle_split = ShuffleSplit(test_size=0.25, n_splits=100)
# set up search
grid_search=GridSearchCV(dtr,param_grid,cv=shuffle_split, 
                              return_train_score=True)
# implement search
grid_search.fit(XOrd,yExtra)
# move results into DataFrame
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','mean_train_score','param_max_depth']])

# Print best params and model
print("best param:",grid_search.best_params_)
print("best model:",grid_search.best_estimator_)
print("best test score:",grid_search.best_score_)

   rank_test_score  mean_test_score  mean_train_score param_max_depth
0               10         0.291889          0.294439               1
1                9         0.468518          0.476640               2
2                8         0.802688          0.811142               3
3                3         0.903427          0.910834               4
4                1         0.913152          0.932145               5
5                2         0.910184          0.945818               6
6                4         0.895301          0.957980               7
7                5         0.881410          0.970556               8
8                6         0.871820          0.980088               9
9                7         0.863249          0.987309              10
best param: {'max_depth': 5}
best model: DecisionTreeRegressor(max_depth=5)
best test score: 0.9131521734131822


As one advantage of decision tree model,optimal feature is selected to split the data and non-informative features will not be selected,there is no big diffrence between using X and XOrd.

## Part 7:Insert interaction term for regression

Add the following feature to your set of features.  This is known as an interaction term, and can help the linear models better target males living in the northwest.

Repeat the monte-carlo cross validations from part (1), but now use (X,yExtra), (XOrd,yExtra), and (Xinteract,yExtra) so that you can compare all three of these.


In [42]:
Xinteract = X.copy()
Xinteract['interact'] = X['region_northwest']*X['sex_male']
#X now only contain data of male who from northwest since northwest and male are represent by dummy variable 1

In [43]:
nmc = 100
lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=nmc, test_size=.25)
CVInfo = cross_validate(lr, X,yExtra, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.6506045432869422
0.6382463655217608


In [44]:
nmc = 100
lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=nmc, test_size=.25)
CVInfo = cross_validate(lr, XOrd,yExtra, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.4383432029593111
0.4188190735387724


In [45]:
nmc = 100
lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=nmc, test_size=.25)
CVInfo = cross_validate(lr, Xinteract,yExtra, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.86931575054181
0.8652525631412906


## Part 8: Compare different models

How did the the (Xinteract,yExtra) regression do relative to the tree, and the Kernel Ridge?

Is it still the best of the linear models?  

Could their be more structure in the data, than what we added in the yExtra creation?

(Xinteract,yExtra) regression results in train score:0.8693 and test score:0.8652
 The tree model using X,yExtra and depth 5 results in highest test score:0.9134 
 The tree model using XOrd,yExtra and depth 5 results in highest test score:0.9131
 The Kernel Ridge model using XOrd,yExtra results in highest test score: 0.9078
 
 (Xinteract,yExtra) regression did not outperform the tree and Kernel Ridge given the result above, but it results in highest tain and test score compared with linear models that use XOrd,yExtra and X,yExtra.

I do not think there could be more structure in the data than when we added in the yExtra creation for the (Xinteract, yExtra) pair. I think that Linear Regression performance increase for the (Xinteract, yExtra) pair because the interaction term in Xinteract is the same interaction term used in the creation of yExtra.